In [1]:
import pandas as pd 

In [2]:
import pandas as pd

df = pd.read_json("hf://datasets/toughdata/quora-question-answer-dataset/Quora-QuAD.jsonl", lines=True)

In [3]:
df

,question,answer
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...
...,...,...
56397,"Alexandria Ocasio-Cortez said ""Going by track ...","I think she’s right, one is a homosexual with ..."
56398,Is becoming a doctor financially worth it?,Yes if you want to help people and eliminate p...
56399,Where can one find the best biryani in bangalore?,Biryani crafts.These guys will give proper aut...
56400,Which smartphone is best for middle class people?,Oneplus nord\n[LINKED_TEXT: https://latesttech...


In [4]:
df.shape

(56402, 2)

In [5]:
def tokenize(text):

  text=text.lower()
  text=text.replace("?","")
  return text.split()

In [6]:
tokenize("Why whenever I get in the shower my girlfriend want to join?")

['why',
 'whenever',
 'i',
 'get',
 'in',
 'the',
 'shower',
 'my',
 'girlfriend',
 'want',
 'to',
 'join']

In [7]:
df.isnull().sum()

question    0
answer      0
dtype: int64

In [8]:
import re

def remove_url(text):
    return re.sub(r'http\S+|www\.\S+', '', text)


In [9]:
df["question"] = df["question"].apply(remove_url)
df["answer"]   = df["answer"].apply(remove_url)


In [10]:
vocab={"<unk>":0}

In [11]:
## make a vocab
def build_vocab(row):
  #print(row["question"],row["answer"])
  tokens_q=tokenize(row["question"])
  tokens_a=tokenize(row["answer"])
  merged_tokens=tokens_q+tokens_a
  #print(merged_tokens)
  ## har 
  for token in merged_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)
    

In [12]:
df.apply(build_vocab,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
56397    None
56398    None
56399    None
56400    None
56401    None
Length: 56402, dtype: object

In [13]:
vocab

{'<unk>': 0,
 'why': 1,
 'whenever': 2,
 'i': 3,
 'get': 4,
 'in': 5,
 'the': 6,
 'shower': 7,
 'my': 8,
 'girlfriend': 9,
 'want': 10,
 'to': 11,
 'join': 12,
 'isn’t': 13,
 'it': 14,
 'awful': 15,
 'you': 16,
 'would': 17,
 'swear': 18,
 'that': 19,
 'there': 20,
 'wasn’t': 21,
 'enough': 22,
 'hot': 23,
 'water': 24,
 'go': 25,
 'around!': 26,
 'what': 27,
 'is': 28,
 'a': 29,
 'proxy,': 30,
 'and': 31,
 'how': 32,
 'can': 33,
 'use': 34,
 'one': 35,
 'proxy': 36,
 'server': 37,
 'system': 38,
 'or': 39,
 'router': 40,
 'provides': 41,
 'gateway': 42,
 'between': 43,
 'users': 44,
 'internet.': 45,
 'therefore,': 46,
 'helps': 47,
 'prevent': 48,
 'cyber': 49,
 'attackers': 50,
 'from': 51,
 'entering': 52,
 'private': 53,
 'network.': 54,
 'server,': 55,
 'referred': 56,
 'as': 57,
 'an': 58,
 '“intermediary”': 59,
 'because': 60,
 'goes': 61,
 'end-users': 62,
 'web': 63,
 'pages': 64,
 'they': 65,
 'visit': 66,
 'online.': 67,
 'when': 68,
 'computer': 69,
 'connects': 70,
 'inte

In [30]:
## no of unique tokens
len(vocab)

339788

In [ ]:
#bahut jyada data hain

SyntaxError: invalid syntax (3291924268.py, line 1)

In [18]:
## convert words to indices 
def words_to_indices(text,vocab):
  indexed_text=[]
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab["<unk>"])
  return indexed_text
  tokens=tokenize(text)

In [17]:
import torch 
from torch.utils.data import Dataset,DataLoader


In [19]:
MAX_ANSWER_LEN = 256

def truncate_sequence(seq, max_len):
    return seq[:max_len] if len(seq) > max_len else seq


In [20]:
class qadataset(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,idx):
    row=self.df.iloc[idx]
    question_indices=words_to_indices(row["question"],self.vocab)
    answer_indices=truncate_sequence(words_to_indices(row["answer"],self.vocab),MAX_ANSWER_LEN)
    return torch.tensor(question_indices),torch.tensor(answer_indices)

In [52]:
for q, a in dataloader:
    print("question:", q.shape)
    print("answer:", a.shape)
    break


question: torch.Size([32, 41])
answer: torch.Size([32, 256])


In [21]:
words_to_indices("hello how are you",vocab)

[2254, 32, 161, 16]

In [22]:
dataset=qadataset(df,vocab)

In [23]:
dataset[0]

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 tensor([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 11, 25, 26]))

In [24]:
from torch.nn.utils.rnn import pad_sequence
import torch

def collate_fn(batch):
    questions = [item[0] for item in batch]   # extract question tensors
    answers = [item[1] for item in batch]     # extract answer tensors

    # pad questions
    questions_padded = pad_sequence(
        questions,
        batch_first=True,
        padding_value=0
    )

    # pad answers
    answers_padded = pad_sequence(
        answers,
        batch_first=True,
        padding_value=0
    )

    return questions_padded, answers_padded


In [25]:
dataloader=DataLoader(dataset,batch_size=32,collate_fn=collate_fn,shuffle=True)

In [26]:
for batch_idx, (questions, answers) in enumerate(dataloader):
    print(f"Batch {batch_idx+1}:")
    print("Questions shape:", questions.shape)
    print("Answers shape:", answers.shape)
    break   # just check first batch


Batch 1:
Questions shape: torch.Size([32, 27])
Answers shape: torch.Size([32, 256])


In [27]:
len(dataloader)

1763

In [ ]:
for question,answer in dataloader:
  print(question,answer)[0]

tensor([[  27,  303, 2246,  ...,    0,    0,    0],
        [  27,   28,    6,  ...,    0,    0,    0],
        [  27,  161,  265,  ...,    0,    0,    0],
        ...,
        [   1,   28,    6,  ...,    0,    0,    0],
        [ 303,  165, 2232,  ...,    0,    0,    0],
        [ 221,   20,  161,  ...,    0,    0,    0]]) tensor([[  300, 91753,   255,  ...,     0,     0,     0],
        [ 2254, 97762,   441,  ...,     0,     0,     0],
        [    6,   252,    28,  ...,     0,     0,     0],
        ...,
        [   14, 20854,   414,  ...,     0,     0,     0],
        [ 1068, 64346,    28,  ...,     0,     0,     0],
        [ 4108,    29,   809,  ...,     0,     0,     0]])
tensor([[   33,    16,   419,  ...,     0,     0,     0],
        [   28,    14,  1931,  ...,     0,     0,     0],
        [   27,  5011,  4282,  ...,     0,     0,     0],
        ...,
        [   32,   303,     3,  ...,     0,     0,     0],
        [  221,     6, 16327,  ...,     0,     0,     0],
        [

In [47]:
## now lets buildrnn model
import torch.nn as nn
import torch

class RNNModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, 128)

        # batch_first=True is VERY important
        self.rnn = nn.RNN(128,256,batch_first=True)
           
        

        self.linear = nn.Linear(256, vocab_size)

    def forward(self, question):
        # question shape: (batch, seq_len)
        embedded = self.embedding(question)  # (batch, seq_len, 128)

        hidden,final = self.rnn(embedded)
        output=self.linear(final.squeeze(0))
        return output

       


In [39]:
da=nn.Embedding(33699,embedding_dim=128)

In [48]:
model=RNNModel(len(vocab))

In [41]:
dataset[0][0]

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [42]:
da(dataset[0][0]).shape

torch.Size([12, 128])

In [49]:
leraning_rate=0.001
num_epochs=15
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=leraning_rate)

In [44]:
for question,answer in dataloader :
  print(question.shape[1],answer.shape[1])
 

38 256
39 256
37 256
42 256
42 256
44 256
37 256
38 256
42 256
38 256
43 256
41 256
27 256
40 256
41 256
48 256
45 256
28 256
33 256
29 256
41 256
45 256
50 256
44 256
33 256
41 256
39 256
37 256
30 256
42 256
38 256
44 256
45 256
38 256
18 256
34 256
36 256
48 256
32 256
41 256
52 256
40 256
42 256
41 256
42 256
43 256
38 256
42 256
41 256
42 256
30 256
41 256
41 256
30 256
41 256
38 256
33 256
32 256
34 256
41 256
42 256
44 256
37 256
40 256
37 256
46 256
45 256
45 256
33 256
45 256
31 256
41 256
45 256
39 256
41 256
34 256
41 256
38 256
32 256
41 256
27 256
41 256
37 256
41 256
45 256
41 256
41 256
45 256
44 256
33 256
37 256
34 256
45 256
25 256
52 256
39 256
38 256
52 256
36 256
28 256
37 256
44 256
30 256
43 256
28 256
41 256
31 256
37 256
33 256
34 256
43 256
49 256
45 256
26 256
43 256
39 256
38 256
45 256
38 256
41 256
31 256
50 256
50 256
34 256
37 256
38 256
45 256
41 256
39 256
47 256
36 256
39 256
24 256
38 256
41 256
50 256
21 256
42 256
41 256
46 256
48 256
45 256
42 256

In [51]:
for epoch in range(num_epochs):
    total_loss=0
    
        
    for question,answer in dataloader:
        optimizer.zero_grad()
        outputs=model(question)
        loss=criterion(outputs,answer[: 0])

        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    print(f"epoch {epoch+1},loss:{total_loss}")

        

ValueError: Expected input batch_size (32) to match target batch_size (0).